# 一、背景

半监督学习（Semi-Supervised Learning）是指通过大量无标注数据和少量有标注数据完成模型训练，解决具有挑战性的模式识别任务。近几年，随着计算硬件性能的提升和大量大规模标注数据集的开源，基于深度卷积神经网络(Deep Convolutional Neural Networks, DCNNs)的监督学习研究取得了革命性进步。然而，监督学习模型的优异性能要以大量标注数据作为支撑，可现实中获得数量可观的标注数据十分耗费人力物力（例如：获取像素级标注数据）。于是，半监督学习逐渐成为深度学习领域的热门研究方向，只需要少量标注数据就可以完成模型训练过程，更适用于现实场景中的各种任务。

[第三届中国AI+创新创业大赛：半监督学习目标定位竞赛](https://aistudio.baidu.com/aistudio/competition/detail/78)

# 二、数据集介绍

训练数据集包括50,000幅像素级有标注的图像，共包含500个类，每个类100幅图像；
A榜测试数据集包括11,878幅无标注的图像；
B榜测试数据集包括10,989幅无标注的图像。

In [ ]:
#解压数据集至data/目录
!unzip -qo data/data95249/train_50k_mask.zip -d data/
!unzip -oq data/data95249/第一阶段test.zip -d data/
!unzip -oq data/data95249/train_image.zip -d data/
!unzip -oq data/data100087/B榜测试数据集.zip -d data/

# 三、数据分析

In [ ]:
#数据集检查和分析，检查完成后，生成work/data_analyse_and_check.log和work/img_pixel_statistics.pkl
#%cd work/
#!python check.py --data_dir data/ --num_classes 2 
#%cd /home/aistudio/

## 1.训练集统计信息

Image channels statistics
Image channels = [3]

Image size statistics:
max width = 4288  min width = 24  max height = 3456  min height = 30

Label class statistics:
(label class, percentage, total pixel number) = [(0, 0.7167, 5107091274), (255, 0.2833, 2018825997)] 

## 2.验证集统计信息

Image channels statistics
Image channels = [3]

Image size statistics:
max width = 3648  min width = 31  max height = 3264  min height = 48

Label class statistics:
(label class, percentage, total pixel number) = [(0, 0.7186, 1320353167), (255, 0.2814, 517136074)] 

## 3.测试集统计信息

Image channels statistics
Image channels = [3]

Image size statistics:
max width = 4288  min width = 67  max height = 3584  min height = 75
value range: 
img_min_value = [0, 0, 0] 
img_max_value = [255, 255, 255]


## 4.图像集Mean和Std

Count the channel-by-channel mean and std of the image:
mean = [102.70997161 115.73388749 120.92191048]
std = [57.97585899 57.8573095  58.89195734]

In [ ]:
#每个通道像素统计信息
%matplotlib inline
import pickle
import matplotlib.pyplot as plt

path = 'work/img_pixel_statistics.pkl'
with open(path, 'rb') as f:
    percentage, img_value_num = pickle.load(f)

for k in range(len(img_value_num)):
    print('channel = {}'.format(k))
    plt.bar(
        list(range(len(img_value_num[k]))),
        img_value_num[k],
        width=1,
        log=True)
    plt.xlabel('image value')
    plt.ylabel('number')
    plt.title('channel={}'.format(k))
    plt.show()

# 四、解题思路

## 1.半监督分割的思路为两种：self-training和consistency learning。一般来说，self-training是离线处理的过程，而consistency learning是在线处理的。



（1）Self-training

Self-training主要分为3步。

第一步，我们在有标签数据上训练一个模型。

第二步，我们用预训练好的模型，为无标签数据集生成伪标签。

第三步，使用有标注数据集的真值标签，和无标注数据集的伪标签，重新训练一个模型。

![](https://ai-studio-static-online.cdn.bcebos.com/ed18d338ea4a4f5d9c25875c197b841cc7d8c1b6c6ed4683992420df43912f14)


（2）Consistency learning

Consistency learning的核心idea是：鼓励模型对经过不同变换的同一样本有相似的输出。这里“变换”包括高斯噪声、随机旋转、颜色的改变等等。Consistency learning主要有三类做法：mean teacher，CPC，PseudoSeg。

PseudoSeg是google发表在ICLR 2021的工作。

![](https://ai-studio-static-online.cdn.bcebos.com/fcdc3af705b14d3994c6105985cd6923c0814bc7eb4649fc90f56cf57725ff9d)


对输入的图像X做两次不同的数据增强，一种“弱增强”（random crop/resize/flip），一种“强增强”(color jittering)。他们将两个增强后图像输入同一个网络f(θ)，得到两个不同的输出。因为“弱增强”下训练更加稳定，他们用“弱增强”后的图像作为target。

## 2.通过了解半监督分割的工作进展，形成Self-training和Consistency learning结合的思路：训练采用Self-training方法，伪标签的处理采用Consistency learning方法，多次迭代训练，推理时多尺度和翻转增强，最后融合多次最好的结果。





# 五、工作步骤

1.采用Paddleseg作为平台，略作一些修改，适用于数据集训练。

（1）PaddleSeg/paddleseg/datasets/dataset.py

             ####标签处理
             
            label = np.asarray(Image.open(label_path).convert('L'))
            
            
            if max(np.unique(label))>1:
            
                label = label/255.
                
            label = label.astype("int64")
            
            label = label[np.newaxis, :, :]
            
 （2）PaddleSeg/paddleseg/core/predict.py
 
            #保存单通道label图像
            
            results_saved_dir = os.path.join(save_dir, 'results')
            
            results_image_path = os.path.join(results_saved_dir, im_file.rsplit(".")[0] + ".jpg")
            
            mkdir(results_image_path)
            
            pred = pred.astype("float32")*255
            
            pred = pred.astype("uint8")
            
            cv2.imwrite(results_image_path, pred)
            
            #######
 
（2）模型选用在FCN基础上修改的Fcn2，增加了OCRHead，并对两个分割头的结果进行早期平均融合。文件为PaddleSeg/paddleseg/models/fcn2.py

     def __init__(.....
        self.head = FCNHead(
            num_classes,
            backbone_indices,
            backbone_channels,
            channels,
            bias=bias)

        self.Ohead = OCRHead(
            num_classes=num_classes,
            in_channels=backbone_channels)
            
    def forward(self, x):
        avg_list = []
        feat_list = self.backbone(x)
        logit_list = self.head(feat_list)
        logit_list1 = self.Ohead(feat_list)
        avg_logit=(logit_list[0]+logit_list1[0])/2
        avg_list.append(avg_logit)
        #avg_list.append(logit_list1[1])
        return [
            F.interpolate(
                logit,
                paddle.shape(x)[2:],
                mode='bilinear',
                align_corners=self.align_corners) for logit in avg_list
        ]


（3）初次训练采用有监督的数据集进行训练，利用初次训练的模型为测试集生成伪标签，将伪标签加入训练集，进行迭代训练。

fcn2.yml为有监督的数据集训练配置文件， Backbone为加了SE的HRNet_W48，训练一次，学习率为0.01，次数为20000，数据增强是垂直和水平翻转。

fcn2_pl.yml为有伪标签的数据集训练配置文件，模型与有监督训练一致，预训练模型采用有监督训练的最好模型，一般迭代训练三次，次数为40000，学习率每次为前一次的一半左右，分别0.01,0.005,0.003，数据增强是垂直和水平翻转,融合多次推理结果，形成新的伪标签。



## **注：以下过程，保留了有监督训练过程。半监督训练过程，需要手工取消注释。**
   
 

# 六、数据集处理

In [ ]:
import sys
sys.path.append("/home/aistudio/PaddleSeg")
import paddleseg
import paddle
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import random
#设置随机数种子
random.seed(2021)

In [ ]:
#写数据文件
def write_txt(file_name, imgs_path, labels_path=None, mode='train', val_pro=0.2):
    assert mode=="train" or mode=="test", "ERROR:mode must be train or test."
    if mode!="test":
        train_path = []
        for idx, f_path in enumerate(imgs_path):
            for i_path in sorted(os.listdir(f_path)):
                path1 = os.path.join(f_path, i_path) 
                path2 = os.path.join(labels_path[idx], i_path)
                train_path.append((path1, path2, str(idx)))
        
        if val_pro>=0 and val_pro<=1:
            #打乱数据
            random.shuffle(train_path)
            val_len = int(len(train_path)*val_pro)
            val_path = train_path[:val_len]
            train_path = train_path[val_len:]
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n")
            with open(file_name[1], 'w') as f:
                for path in val_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n")  
            return len(train_path), val_len
        else:
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n") 
            return len(train_path), 0
    else:
        with open(file_name, 'w') as f:
            for path in imgs_path:
                img_path = os.path.join(test_path, path)
                f.write(img_path+"\n")

## 6.1生成有监督训练集
第一次训练时，生成有监督训练集

In [10]:
#生成不含伪标注训练集
def create_txt(data_root, train_imgs_dir=None, train_labels_dir=None, test_dir=None, val_pro=0.2):
    if train_imgs_dir is not None:
        if os.path.exists("train.txt"):
            os.remove("train.txt")
        if os.path.exists("val.txt"):
            os.remove("val.txt")
        train_imgs_dir = os.path.join(data_root, train_imgs_dir)
        train_labels_dir = os.path.join(data_root, train_labels_dir)
        file_names = os.listdir(train_imgs_dir)
        file_names = sorted(file_names)
        train_imgs_path, train_labels_path =[], []
        for na in file_names:
            train_imgs_path.append(os.path.join(train_imgs_dir, na))
            train_labels_path.append(os.path.join(train_labels_dir, na))
        train_len, val_len = write_txt(["train.txt", "val.txt"], train_imgs_path, train_labels_path, mode='train', val_pro=val_pro)
        
        print("训练数据整理完毕！训练集长度：{}，验证集长度：{}， 类别数：{}".format(train_len, val_len, len(file_names)))

    if test_dir is not None:
        if os.path.exists("test.txt"):
            os.remove("test.txt")
        global test_path
        test_path = os.path.join(data_root, test_dir)
        test_imgs_path_list = sorted(os.listdir(test_path))
        write_txt("test.txt", test_imgs_path_list, mode="test")
        print("测试数据整理完毕！测试集长度：{}".format(len(test_imgs_path_list)))

In [11]:
#生成不含伪标注的训练文件
data_root = "data"
train_imgs_dir = "train_image"
train_labels_dir = "train_50k_mask"
test_dir = "test_image"
create_txt(data_root, train_imgs_dir, train_labels_dir, test_dir, val_pro=0.2)

## 6.2 生成半监督数据集
迭代训练时，根据前次最好的模型生成伪标注，形成半监督训练集

In [12]:
#利用上次训练模型，生成伪标注，加入数据集
'''
!mkdir data/train_image/pl
!mkdir data/train_50k_mask/pl
!cp data/val_image/* data/train_image/pl/
!cp data/test_image/* data/train_image/pl/
!cp work/vote728/* data/train_50k_mask/pl
!cp work/output_fcn2_pl_msf_4k_0.76667_0805/result_1/results/* data/train_50k_mask/pl
!rename 's/\.JPEG/\.jpg/' data/train_image/pl/*.JPEG
'''

In [13]:
#生成伪标注训练集
'''
def create_pl_txt(data_root, train_imgs_dir=None, train_labels_dir=None, test_dir=None, val_pro=0.2):
    if train_imgs_dir is not None:
        if os.path.exists("train_pl.txt"):
            os.remove("train_pl.txt")
        if os.path.exists("val_pl.txt"):
            os.remove("val_pl.txt")
        train_imgs_dir = os.path.join(data_root, train_imgs_dir)
        train_labels_dir = os.path.join(data_root, train_labels_dir)
        file_names = os.listdir(train_imgs_dir)
        file_names = sorted(file_names)
        train_imgs_path, train_labels_path =[], []
        for na in file_names:
            train_imgs_path.append(os.path.join(train_imgs_dir, na))
            train_labels_path.append(os.path.join(train_labels_dir, na))
        train_len, val_len = write_txt(["train_pl.txt", "val_pl.txt"], train_imgs_path, train_labels_path, mode='train', val_pro=val_pro)
        
        print("训练数据整理完毕！训练集长度：{}，验证集长度：{}， 类别数：{}".format(train_len, val_len, len(file_names)))

    if test_dir is not None:
        if os.path.exists("test_pl.txt"):
            os.remove("test_pl.txt")
        global test_path
        test_path = os.path.join(data_root, test_dir)
        test_imgs_path_list = sorted(os.listdir(test_path))
        write_txt("test_pl.txt", test_imgs_path_list, mode="test")
        print("测试数据整理完毕！测试集长度：{}".format(len(test_imgs_path_list)))
'''

In [14]:
'''
data_root = "data"
train_imgs_dir = "train_image"
train_labels_dir = "train_50k_mask"
test_dir = "test_image"
create_pl_txt(data_root, train_imgs_dir, train_labels_dir, test_dir, val_pro=0.2)
'''

# 七、训练

In [15]:
#有监督训练
!python PaddleSeg/train.py --config fcn2.yml --do_eval --use_vdl --save_dir /home/aistudio/model_fcn2 --save_interval 10000 #--resume_model model_fcn3_pl/iter_60000/

In [16]:
#半监督训练，迭代三次，采用含有伪标注的数据集，需要手工修改为预训练模型为上次训练好的模型，学习率0.01，0.005，0.003，训练40000次
#!python PaddleSeg/train.py --config fcn2_pl.yml --do_eval --use_vdl --save_dir /home/aistudio/model_fcn2_pl --save_interval 10000 #--resume_model model_fcn3_pl/iter_60000/

# 八、推理

In [17]:
#有监督：用不含伪标注的训练模模型推理，推理加翻转和多尺度0.8,1,1.2融合
#!python PaddleSeg/predict.py --config fcn2.yml --model_path model_fcn2/best_model/model.pdparams --image_path data/test_image --save_dir output/result_1 --flip_horizontal --flip_vertical --aug_pred --scales 0.8 1 1.2

In [18]:
#半监督：用含伪标注训练的模模型推理，推理加翻转和多尺度0.8,1,1.2融合
!python PaddleSeg/predict.py --config fcn2_pl.yml --model_path work/model_fcn2_pl_0805/best_model/model.pdparamss --image_path data/test_image --save_dir output/result_1 --flip_horizontal --flip_vertical --aug_pred --scales 0.8 1 1.2

# 九、提交



In [19]:
#单模型结果提交
%cd output/result_1/results
!zip -r -oq /home/aistudio/predb.zip ./
%cd /home/aistudio

^C


zip error: Interrupted (aborting)
/home/aistudio


In [20]:
#多阶段模型结果融合
#!python work/vote.py

In [21]:
#%cd work/vote0805/
#!zip -r -oq /home/aistudio/work/vote0805.zip ./
#%cd /home/aistudio/work

# 总结

   此次参赛，主要是为了学习和研究半监督学习方向的最新工作进展，读了一些论文，并在比赛中进行实验，收获良多。
   
   半监督学习采用多种方法相结合，形成端到端的方法，是最有前景的研究和应用方向。
   
  
